In [102]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import gzip
import os

In [103]:
docword_file_base_name = 'docword.{}.txt.gz'
unzipped_docword_file_base_name = 'docword.{}.txt'
vocab_file_base_name = 'vocab.{}.txt'

def check_file_existence(path):
    found = os.path.isfile(path)
    
    if not found:
        print("File {} not found.".format(path))
    return found

def get_data_set(requested_dataset, \
               base_url='https://archive.ics.uci.edu/ml/machine-learning-databases/bag-of-words/'):
    
    
    for data_set in requested_dataset:
        
        # Vocabulary file.
        with urlopen(base_url + vocab_file_base_name.format(data_set)) as response:
            data = response.read()
            encoding = response.headers.get_content_charset('utf-8')
            text = data.decode(encoding).splitlines() 

    return text    

In [104]:
# Check if the docword exists in the specific folder.

folder_name = 'docword_files/'
file_name = unzipped_docword_file_base_name.format('enron')
path = folder_name+file_name
check_file_existence(path)

True

In [105]:
requested_dataset = ['enron']

# Get the dataset.
data = get_data_set(requested_dataset)

In [107]:
# Load the vocabulary into the dataframe.
df = pd.DataFrame(data=data, columns=['word'])

# Shift the index by one.
df.index += 1

# Rename the index column.
df.index.name='word_id'

In [109]:
def populate_doc_word(df,data_set_name):
    ''' This method is used to populate the dataframe df with the associated docword information.
    data_set_name is the name used to retrieve the docword file. '''
    
    folder_name = 'docword_files/'
    file_name = unzipped_docword_file_base_name.format(data_set_name)
    path = folder_name+file_name
    index = 0
    
    info = dict()
    
    # Empty columns.
    df['document_id'] = ""
    df['count'] = ""
    
    if check_file_existence(path):
        print('File found')
        with open(path, 'rb') as file:
            for line in file:
                if index == 0:
                    info['docs_num'] = line
                elif index == 1:
                    info['words_num'] = line
                elif index == 2:
                    info['non_zero_num'] = line
                else:
                    # Decode the current line.
                    line = line.decode('utf-8').split()
                    
                    # Insert the information into the relative column of the dataframe.
                    df["document_id"].iloc[int(line[0])-1] = line[1]
                    df["count"].iloc[int(line[0])-1] = line[2]
                index += 1
    
    return info

In [110]:
populate_doc_word(df,'enron')

File found


KeyboardInterrupt: 

In [127]:
# df["document_id"].iloc[0] = 5
df.index(1)

TypeError: 'RangeIndex' object is not callable